## 뉴스 카테고리 다중분류 [프로젝트]

In [2]:
import tensorflow
import matplotlib
import seaborn 
import numpy 
import pandas
import sklearn

print(tensorflow.__version__)
print(matplotlib.__version__)
print(seaborn.__version__)
print(numpy.__version__)
print(pandas.__version__)
print(sklearn.__version__)

2.19.0
3.9.2
0.13.2
1.26.4
2.2.2
1.5.1


In [3]:
from tensorflow.keras.datasets import reuters

In [5]:
(x_train, y_train), (x_test, y_test) = reuters.load_data(num_words=None, test_split=0.2)

In [ ]:
# (x_train, y_train), (x_test, y_test) = reuters.load_data(num_words=5000, test_split=0.2)

In [8]:
# 원본 뉴스 데이터로 복원하기

word_index = reuters.get_word_index(path="reuters_word_index.json")
index_to_word = {index+3 : word for word, index in word_index.items()}

for index, token in enumerate(("<pad>", "<sos>", "<unk>")):
  index_to_word[index]=token

decoded = []
for i in range(len(x_train)):
    t = ' '.join([index_to_word[index] for index in x_train[i]])
    decoded.append(t)

x_train = decoded

decoded_test = []
for i in range(len(x_test)):
    t = ' '.join([index_to_word[index] for index in x_test[i]])
    decoded_test.append(t)

x_test = decoded_test

In [9]:
x_train[:5]

['<sos> mcgrath rentcorp said as a result of its december acquisition of space co it expects earnings per share in 1987 of 1 15 to 1 30 dlrs per share up from 70 cts in 1986 the company said pretax net should rise to nine to 10 mln dlrs from six mln dlrs in 1986 and rental operation revenues to 19 to 22 mln dlrs from 12 5 mln dlrs it said cash flow per share this year should be 2 50 to three dlrs reuter 3',
 "<sos> generale de banque sa lt genb br and lt heller overseas corp of chicago have each taken 50 pct stakes in factoring company sa belgo factors generale de banque said in a statement it gave no financial details of the transaction sa belgo factors' turnover in 1986 was 17 5 billion belgian francs reuter 3",
 '<sos> shr 3 28 dlrs vs 22 cts shr diluted 2 99 dlrs vs 22 cts net 46 0 mln vs 3 328 000 avg shrs 14 0 mln vs 15 2 mln year shr 5 41 dlrs vs 1 56 dlrs shr diluted 4 94 dlrs vs 1 50 dlrs net 78 2 mln vs 25 9 mln avg shrs 14 5 mln vs 15 1 mln note earnings per share reflect th

In [10]:
x_test[:5]

['<sos> the great atlantic and pacific tea co said its three year 345 mln dlr capital program will be be substantially increased to accommodate growth and expansion plans for waldbaum inc and shopwell inc over the next two years a and p said the acquisition of shopwell in august 1986 and waldbaum in december helped us achieve better than expected results in the fourth quarter ended february 28 its net income from continuing operations jumped 52 6 pct to 20 7 mln dlrs or 55 cts a share in the latest quarter as sales increased 48 3 pct to 1 58 billion dlrs a and p gave no details on the expanded capital program but it did say it completed the first year of the program during 1986 a and p is 52 4 pct owned by lt tengelmann warenhandelsgesellschaft of west germany reuter 3',
 "<sos> philippine sugar production in the 1987 88 crop year ending august has been set at 1 6 mln tonnes up from a provisional 1 3 mln tonnes this year sugar regulatory administration sra chairman arsenio yulo said yu

In [11]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

In [12]:
dtmvector = CountVectorizer()
x_train_dtm = dtmvector.fit_transform(x_train)
print(x_train_dtm.shape)

(8982, 26506)


In [13]:
tfidf_transformer = TfidfTransformer()
tfidfv = tfidf_transformer.fit_transform(x_train_dtm)
print(tfidfv.shape)

(8982, 26506)


In [14]:
from sklearn.naive_bayes import MultinomialNB # 다항분포 나이브 베이즈 모델
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.naive_bayes import ComplementNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score # 정확도 계산

In [15]:
# 나이브 베이즈 분류기

model = MultinomialNB()
model.fit(tfidfv, y_train) # 모델로 학습

MultinomialNB()

In [16]:
x_test_dtm = dtmvector.transform(x_test) # 테스트데이터 DTM 변환
tfidfv_test = tfidf_transformer.transform(x_test_dtm) # DTM을 TF-IDF 행렬로 변환

predicted = model.predict(tfidfv_test) # 모델로 테스트데이터 예측
print("정확도: ", accuracy_score(y_test, predicted)) # 예측값과 실제값 비교

정확도:  0.5997328584149599


In [18]:
# Complement Naive Bayes Classifier(CNB) 모델 # 데이터의 불균형을 고려하여 가중치를 부여

cb = ComplementNB()
cb.fit(tfidfv, y_train)

predicted = cb.predict(tfidfv_test)
print("정확도: ", accuracy_score(y_test, predicted))

정확도:  0.7649154051647373


In [19]:
# 로지스틱 회귀(Logistic Regression)
lr = LogisticRegression(C=10000, penalty='l2', max_iter=3000)
lr.fit(tfidfv, y_train)

predicted = lr.predict(tfidfv_test) #테스트 데이터에 대한 예측
print("정확도:", accuracy_score(y_test, predicted)) #예측값과 실제값 비교

정확도: 0.811219946571683


In [20]:
# LinearSVC

lsvc = LinearSVC(C=1000, penalty='l1', max_iter=3000, dual=False)
lsvc.fit(tfidfv, y_train)

predicted = lsvc.predict(tfidfv_test) #테스트 데이터에 대한 예측
print("정확도:", accuracy_score(y_test, predicted)) #예측값과 실제값 비교

정확도: 0.7916295636687445


c:\Users\User\anaconda3\Lib\site-packages\sklearn\svm\_base.py:1235: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


In [21]:
# 결정 트리
tree = DecisionTreeClassifier(max_depth=10, random_state=0)
tree.fit(tfidfv, y_train)

predicted = tree.predict(tfidfv_test) #테스트 데이터에 대한 예측
print("정확도:", accuracy_score(y_test, predicted)) #예측값과 실제값 비교

정확도: 0.6211041852181657


In [22]:
# 랜덤 포레스트

forest = RandomForestClassifier(n_estimators = 5, random_state = 0)
forest.fit(tfidfv, y_train)

predicted = forest.predict(tfidfv_test) #테스트 데이터에 대한 예측
print("정확도:", accuracy_score(y_test, predicted)) #예측값과 실제값 비교

정확도: 0.6544968833481746


In [23]:
# 그래디언트 부스팅 트리

grbt = GradientBoostingClassifier(random_state=0, verbose=3) 
grbt.fit(tfidfv, y_train)

predicted = grbt.predict(tfidfv_test) #테스트 데이터에 대한 예측
print("정확도:", accuracy_score(y_test, predicted)) #예측값과 실제값 비교

      Iter       Train Loss   Remaining Time 
         1           1.4301           20.39m
         2       76760.8825           19.43m
         3   563875761.7725           20.39m
         4 660851364965680256.0000           20.81m
         5 1012265032800504393719904542524107198673282943089732315820038266473352642847664146935533507074646775800850487562408087411539050496.0000           20.33m
         6 1012265032800504393719904542524107198673282943089732315820038266473352642847664146935533507074646775800850487562408087411539050496.0000           20.28m
         7 1012265032800504393719904542524107198673282943089732315820038266473352642847664146935533507074646775800850487562408087411539050496.0000           21.33m
         8 1012265032800504393719904542524107198673282943089732315820038266473352642847664146935533507074646775800850487562408087411539050496.0000           22.63m
         9 1012265032800504393719904542524107198673282943089732315820038266473352642847664146935533507074646775

In [24]:
# 보팅

lr = LogisticRegression(C=10000, penalty='l2', max_iter=3000)
cb = ComplementNB()
grbt = GradientBoostingClassifier(random_state=0) 

voting_classifier = VotingClassifier(estimators=[
        ('lr', lr), ('cb', cb), ('grbt', grbt)], voting='soft')
voting_classifier.fit(tfidfv, y_train)

predicted = voting_classifier.predict(tfidfv_test) #테스트 데이터에 대한 예측
print("정확도:", accuracy_score(y_test, predicted)) #예측값과 실제값 비교

정확도: 0.8161175422974176
